<a href="https://colab.research.google.com/github/LuanDavid7/AI_EKT/blob/main/AI_Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests

# Configurações da API
API_KEY = "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"  # Sua chave do Groq
MODEL = "llama3-70b-8192"  # Modelo Llama 3 70B (ou "llama3-8b-8192" para versão leve)
URL = "https://api.groq.com/openai/v1/chat/completions"

def chat_llama3():
    print("\n🔵 Bem-vindo ao Chat Llama 3! (Digite 'sair' para encerrar)")
    historico = []

    while True:
        # Pega a pergunta do usuário
        pergunta = input("\nVocê: ")

        if pergunta.lower() in ["sair", "exit"]:
            print("🟡 Encerrando chat...")
            break

        # Adiciona ao histórico (opcional, para contexto)
        historico.append({"role": "user", "content": pergunta})

        # Envia para a API
        resposta = requests.post(
            URL,
            headers={"Authorization": f"Bearer {API_KEY}"},
            json={
                "model": MODEL,
                "messages": historico,
                "temperature": 0.7
            }
        )

        # Processa a resposta
        if resposta.status_code == 200:
            resposta_ia = resposta.json()["choices"][0]["message"]["content"]
            print(f"\nLlama 3: {resposta_ia}")
            historico.append({"role": "assistant", "content": resposta_ia})  # Atualiza histórico
        else:
            print(f"🔴 Erro: {resposta.text}")

# Inicia o chat
if __name__ == "__main__":
    chat_llama3()


🔵 Bem-vindo ao Chat Llama 3! (Digite 'sair' para encerrar)


KeyboardInterrupt: Interrupted by user

In [ ]:
import requests

# Configurações da API
API_KEY = "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"  # Sua chave do Groq
MODEL = "llama3-70b-8192"  # Modelo Llama 3 70B (ou "llama3-8b-8192" para versão leve)
URL = "https://api.groq.com/openai/v1/chat/completions"

def chat_llama3():
    print("\n🔵 Bem-vindo à Neoenergia Elektro! Digite 'sair' para encerrar.")
    historico = []

    # Define o papel inicial da IA
    historico.append({"role": "system", "content": """
Você é um atendente da Neoenergia Elektro, uma distribuidora de energia.
Seu objetivo é ajudar os clientes a resolverem seus problemas de energia com empatia e eficiência.
Busque compreender o problema real do cliente, mesmo que a descrição seja confusa.
Pergunte perguntas esclarecedoras para entender a situação e oferecer a melhor solução.
Apresente-se como um atendente da Elektro e identifique a si mesmo como o atendente responsável pelo chamado.
Os problemas mais comuns são: falta de energia (geral, específica, falta de fase, sigfi), risco de morte (fio partido, poste abalroado), níveis de tensão (baixa, oscilação), e defeitos (furto, falha em ramal, medidor, objeto estranho na rede).
Você deve manter um tom profissional, amável, e eficiente.
"""})

    while True:
        pergunta = input("\nVocê: ")

        if pergunta.lower() in ["sair", "exit"]:
            print("🟡 Encerrando atendimento...")
            break

        historico.append({"role": "user", "content": pergunta})

        resposta = requests.post(
            URL,
            headers={"Authorization": f"Bearer {API_KEY}"},
            json={
                "model": MODEL,
                "messages": historico,
                "temperature": 0.7
            }
        )

        if resposta.status_code == 200:
            resposta_ia = resposta.json()["choices"][0]["message"]["content"]
            print(f"\nAtendente Elektro: {resposta_ia}")
            historico.append({"role": "assistant", "content": resposta_ia})
        else:
            print(f"🔴 Erro: {resposta.text}")

if __name__ == "__main__":
    chat_llama3()


In [9]:
import requests
import json
import re
import time

# API Configuration
API_KEY = "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"
MODEL = "llama3-70b-8192"  # Using Llama 3 70B (can change to "llama3-8b-8192" for lighter version)
URL = "https://api.groq.com/openai/v1/chat/completions"

# Common issue types and subtypes for classification
ISSUE_TYPES = {
    "FALTA DE ENERGIA": ["FALTA DE ENERGIA GERAL", "FALTA DE ENERGIA", "FALTA DE FASE",
                         "SIGFI FALTA DE ENERGIA FOTOVOLTAICO", "FALTA DE FASE GERAL",
                         "Falta de Energia em Medidor Eletronico", "Falta de Fase em Medidor Eletrônico"],

    "RISCO DE MORTE": ["OUTROS", "FIO PARTIDO NA REDE PRIMARIA", "FIO PARTIDO NO RAMAL DE SERVICO",
                       "FIO PARTIDO NA REDE SECUNDARIA", "POSTE ABALROADO SEM FIO PARTIDO",
                       "POSTE ABALROADO COM FIO PARTIDO"],

    "PROVAVEL DEFEITO FALHA": ["FURTO FIACÃO/PADRÃO", "FALHA EM RAMAL DE SERVICO",
                              "FIO DANDO FOGO NO RAMAL DE SERVICO", "FALHA EM MEDIDOR",
                              "OBJETO ESTRANHO NA REDE SECUNDARIA", "OBJETO ESTRANHO NA REDE PRIMARIA",
                              "POSTE PADR?O QUEBRADO", "FIO DANDO FOGO NA REDE PRIMARIA",
                              "FIO DANDO FOGO COM ARVORE NA REDE", "FURTO EQUIPAMENTO",
                              "ELO FUSIVEL QUEIMADO/CAIDO", "FIO DANDO FOGO NA REDE SECUNDARIA",
                              "CHOQUE EM INSTALAC?O", "FALHA EM QUICK-LAG", "FURTO REDE",
                              "INVERSÃO DE FASE"],

    "NIVEIS DE TENSÃO": ["TENSÃO BAIXA", "OSCILAC?O DE TENSÃO", "TENSÃO ALTA"],

    "OUTROS": ["TRANSFORMADOR"],

    "EMERGENCIA MEDICA": ["HOSPITAL COM PROBLEMA", "EQUIPAMENTO SUPORTE A VIDA"]
}

# Investigation questions to determine if it's an internal problem
INVESTIGATION_QUESTIONS = {
    "FALTA DE ENERGIA": [
        "Você poderia me informar se a falta de energia afeta apenas sua residência ou se os vizinhos também estão sem energia?",
        "Você já verificou o quadro de disjuntores interno da sua residência para ver se algum está desarmado?",
        "Verificou também o disjuntor geral no padrão de entrada (relógio/medidor de energia)?",
        "Notou alguma ocorrência como raios, ventos fortes ou acidentes próximos antes da falta de energia?"
    ],
    "FALTA DE FASE": [
        "Alguns aparelhos funcionam normalmente enquanto outros não ligam?",
        "Todas as tomadas da casa estão funcionando ou apenas algumas?",
        "Você já verificou se todos os disjuntores do quadro de energia estão ligados?",
        "Notou alguma queda de energia ou oscilação antes do problema começar?"
    ],
    "PROVAVEL DEFEITO FALHA": [
        "Onde exatamente você está observando o problema? É na entrada de energia (medidor/poste) ou dentro da residência?",
        "Você verificou todos os disjuntores do quadro de energia interno?",
        "Está ocorrendo algum cheiro de queimado ou ruído estranho? Pode me dizer onde?",
        "O problema começou após alguma tempestade ou ventania?"
    ],
    "NIVEIS DE TENSÃO": [
        "Você percebe lâmpadas piscando ou mais fracas que o normal?",
        "Os aparelhos eletrônicos estão funcionando mais devagar ou desligando sozinhos?",
        "O problema ocorre em toda a vizinhança ou apenas na sua residência?",
        "Já realizou alguma modificação recente na instalação elétrica da sua casa?"
    ],
    "DEFAULT": [
        "Você já verificou o quadro de disjuntores interno da sua residência?",
        "Verificou também o disjuntor geral no padrão de entrada (relógio/medidor)?",
        "Os seus vizinhos estão enfrentando o mesmo problema?",
        "O problema começou após alguma mudança na rede elétrica ou tempestade?"
    ]
}

# Function to determine responsibility based on investigation
def determine_responsibility(issue_type, responses):
    """
    Determine if the problem is internal (client responsibility) or external (distributor responsibility)
    based on the answers to investigation questions.

    Args:
        issue_type: The type of issue reported
        responses: Dictionary of question responses

    Returns:
        is_internal: Boolean indicating if it's an internal problem
        certainty: Level of certainty (high, medium, low)
        reason: Explanation for the decision
    """
    is_internal = False
    certainty = "low"
    reason = "Não foi possível determinar com certeza se o problema é interno ou externo."

    # Check for clear indicators in the responses
    if "vizinhos" in responses and responses["vizinhos"]:
        if "não" in responses["vizinhos"].lower() and "também" not in responses["vizinhos"].lower():
            # Only client affected, likely internal
            is_internal = True
            certainty = "high"
            reason = "Apenas a sua residência está sem energia, enquanto os vizinhos têm energia normal."
        elif "sim" in responses["vizinhos"].lower() or "também" in responses["vizinhos"].lower():
            # Neighbors also affected, external problem
            is_internal = False
            certainty = "high"
            reason = "Os vizinhos também estão enfrentando o mesmo problema, indicando um problema na rede de distribuição."

    # Check disjunctor status
    if "disjuntores" in responses and responses["disjuntores"]:
        if "caído" in responses["disjuntores"].lower() or "desarmado" in responses["disjuntores"].lower() or "desligado" in responses["disjuntores"].lower():
            is_internal = True
            certainty = "high"
            reason = "Há disjuntores desarmados no quadro interno, o que indica um problema na instalação interna."

    # If problem is in the external meter/pole
    if "localização" in responses and responses["localização"]:
        if "poste" in responses["localização"].lower() or "rua" in responses["localização"].lower() or "antes do medidor" in responses["localização"].lower():
            is_internal = False
            certainty = "high"
            reason = "O problema está localizado no poste, na rua ou antes do medidor, que são responsabilidade da distribuidora."
        elif "dentro" in responses["localização"].lower() or "após o medidor" in responses["localização"].lower() or "depois do relógio" in responses["localização"].lower():
            is_internal = True
            certainty = "high"
            reason = "O problema está localizado dentro da residência ou após o medidor/relógio, que é responsabilidade do cliente."

    # If there are visible damages to the distributor's equipment
    if "danos_visíveis" in responses and responses["danos_visíveis"]:
        if "poste" in responses["danos_visíveis"].lower() or "fio caído" in responses["danos_visíveis"].lower() or "transformador" in responses["danos_visíveis"].lower():
            is_internal = False
            certainty = "high"
            reason = "Há danos visíveis na rede externa (poste, fios ou transformador), que são responsabilidade da distribuidora."

    # If the investigation wasn't conclusive, make educated guess based on the issue type
    if certainty == "low":
        if issue_type == "RISCO DE MORTE":
            is_internal = False
            certainty = "medium"
            reason = "Situações de risco na rede elétrica geralmente são responsabilidade da distribuidora."
        elif issue_type == "EMERGENCIA MEDICA":
            is_internal = False
            certainty = "medium"
            reason = "Emergências médicas envolvendo energia são tratadas como prioridade da distribuidora."

    return is_internal, certainty, reason

# Function to generate a unique protocol number
def generate_protocol():
    timestamp = int(time.time())
    return f"NEO-{timestamp}-{int(timestamp/13) % 1000:03d}"

# Function to send message to AI model
def send_to_ai(prompt, conversation_history=None):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    system_prompt = """Você é um atendente virtual da Neoenergia Elektro, uma distribuidora de energia elétrica.
    Seu nome é Maria e você deve atender aos clientes com muita empatia e compreensão.

    IMPORTANTE:
    1. Você deve investigar adequadamente para determinar se o problema relatado é de responsabilidade da distribuidora ou do cliente
    2. Faça perguntas específicas para distinguir entre problemas internos (após o medidor) e externos (rede da distribuidora)
    3. SEMPRE peça ao cliente se ele já verificou os disjuntores do quadro interno e do padrão de entrada antes de concluir o diagnóstico
    4. Gere protocolo de atendimento APENAS para problemas que são responsabilidade da distribuidora
    5. Para problemas internos (após o medidor/relógio), oriente o cliente mas NÃO gere protocolo

    Seja sempre cordial, paciente e demonstre preocupação genuína com o problema do cliente.
    Use uma linguagem clara e acessível, evitando termos técnicos complexos.
    Forneça orientações de segurança quando necessário.
    """

    messages = [{"role": "system", "content": system_prompt}]

    # Add conversation history if provided
    if conversation_history:
        messages.extend(conversation_history)

    # Add the current prompt
    messages.append({"role": "user", "content": prompt})

    data = {
        "model": MODEL,
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 800
    }

    try:
        response = requests.post(URL, headers=headers, data=json.dumps(data))
        if response.status_code == 200:
            return response.json()["choices"][0]["message"]["content"]
        else:
            return f"Erro na API: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Erro ao conectar com a API: {str(e)}"

# Updated main function for interactive investigation
def handle_customer_complaint():
    print("=== Atendimento Virtual Neoenergia Elektro ===")
    print("Digite 'sair' para encerrar o atendimento\n")

    # Initial greeting
    print("Maria: Olá! Sou Maria, assistente virtual da Neoenergia Elektro. Como posso ajudar você hoje?")

    conversation_history = []
    investigation_complete = False
    is_internal_problem = None

    while True:
        user_input = input("\nVocê: ")
        if user_input.lower() in ['sair', 'exit', 'quit']:
            print("\nMaria: Obrigada por entrar em contato com a Neoenergia Elektro. Tenha um ótimo dia!")
            break

        # Store the user message in history
        conversation_history.append({"role": "user", "content": user_input})

        # First message - detect initial issue type
        if len(conversation_history) == 1:
            # Get initial issue assessment
            initial_issue_type = identify_initial_issue_type(user_input)

            # Generate investigation prompt based on the issue type
            investigation_questions = INVESTIGATION_QUESTIONS.get(
                initial_issue_type, INVESTIGATION_QUESTIONS["DEFAULT"]
            )

            investigation_prompt = f"""
O cliente relatou: "{user_input}"

Com base nesse relato inicial, identifiquei que pode ser um problema de {initial_issue_type}.

IMPORTANTE: Antes de determinar se é um problema interno ou da distribuidora, preciso investigar melhor.

Por favor, faça as seguintes perguntas ao cliente de forma conversacional (uma de cada vez, não liste todas):
1. Pergunte se o problema afeta apenas a residência dele ou também os vizinhos
2. Pergunte se o cliente já verificou os disjuntores no quadro interno da residência
3. Pergunte se verificou o disjuntor geral no padrão de entrada (medidor/relógio)
4. Pergunte se há algum sinal visível de problema (como fios caídos, cheiro de queimado, etc.)

Seja empático e explique que essas perguntas ajudarão a identificar o tipo de problema e direcionar o melhor atendimento.
"""

            response = send_to_ai(investigation_prompt)

        # Investigation phase
        elif not investigation_complete:
            # Get more context about previous conversation
            context_prompt = f"""
Analise a conversa atual com o cliente e continue a investigação para determinar se o problema é interno (responsabilidade do cliente) ou externo (responsabilidade da distribuidora).

Mensagem mais recente do cliente: "{user_input}"

IMPORTANTE:
1. Se o cliente respondeu a uma pergunta sobre verificação de disjuntores, vizinhos afetados, ou localização do problema, use essa informação
2. Se ainda não tiver perguntado sobre os disjuntores do quadro interno e do padrão de entrada, pergunte isso agora
3. Se já tiver informações suficientes para determinar se é problema interno ou externo, conclua a investigação e:
   a) Para problemas internos: Explique ao cliente que é responsabilidade dele e oriente sobre os próximos passos
   b) Para problemas da distribuidora: Gere um protocolo e explique o processo de atendimento

Mantenha a conversa natural e empática.
"""

            # Add the full conversation history
            full_history = []
            for msg in conversation_history:
                full_history.append(msg)

            response = send_to_ai(context_prompt, full_history[:-1])  # Excluding the last message which we included in the prompt

            # Check if the response indicates completion of investigation
            if "protocolo" in response.lower() or "eletricista particular" in response.lower() or "responsabilidade do cliente" in response.lower() or "nossa equipe" in response.lower():
                investigation_complete = True

        # After investigation is complete
        else:
            follow_up_prompt = f"""
A investigação do problema já foi concluída. O cliente está perguntando: "{user_input}"

Responda à pergunta do cliente mantendo a coerência com o que já foi determinado anteriormente sobre o tipo de problema e quem é o responsável.

Se o cliente estiver perguntando sobre o status do atendimento ou outra informação relacionada, forneça as orientações adequadas.
"""
            response = send_to_ai(follow_up_prompt, conversation_history[:-1])

        # Store assistant's response in history
        conversation_history.append({"role": "assistant", "content": response})

        print(f"\nMaria: {response}")

# Function to identify initial issue type from first message
def identify_initial_issue_type(user_input):
    user_input = user_input.upper()

    # First try to find exact matches
    for type_name, subtypes in ISSUE_TYPES.items():
        for subtype in subtypes:
            if subtype.upper() in user_input:
                return type_name

    # If no exact match, look for keywords
    keywords = {
        "FALTA DE ENERGIA": ["sem luz", "acabou a luz", "energia caiu", "não tem energia", "sem energia", "sem eletricidade"],
        "RISCO DE MORTE": ["fio caído", "fio na rua", "poste quebrado", "fio partido", "risco", "perigo", "acidente"],
        "PROVAVEL DEFEITO FALHA": ["faísca", "curto", "queimando", "cheiro", "defeito", "falha", "problema", "quebrou"],
        "NIVEIS DE TENSÃO": ["tensão", "oscilação", "baixa voltagem", "alta voltagem", "queimou aparelho", "piscando"],
        "EMERGENCIA MEDICA": ["hospital", "clínica", "aparelho médico", "respirador", "equipamento", "vida", "paciente"],
        "OUTROS": ["transformador", "caixa de luz", "poste"]
    }

    for type_name, type_keywords in keywords.items():
        for keyword in type_keywords:
            if keyword.upper() in user_input:
                return type_name

    # Default if no match found
    return "FALTA DE ENERGIA"

# Class-based implementation for more structured conversation flow
class NeoEnergiaAssistant:
    def __init__(self):
        self.conversation_history = []
        self.investigation_stage = 0  # 0: initial, 1-4: investigation questions, 5: conclusion
        self.issue_type = None
        self.is_internal_problem = None
        self.investigation_responses = {}
        self.protocol = None

    def reset(self):
        self.__init__()

    def process_message(self, user_input):
        # Add user message to history
        self.conversation_history.append({"role": "user", "content": user_input})

        # Initial greeting or first message
        if self.investigation_stage == 0:
            # Identify issue type
            self.issue_type = identify_initial_issue_type(user_input)
            self.investigation_stage = 1

            # First investigation question
            investigation_questions = INVESTIGATION_QUESTIONS.get(
                self.issue_type, INVESTIGATION_QUESTIONS["DEFAULT"]
            )

            initial_prompt = f"""
O cliente relatou: "{user_input}"

Baseado nisso, parece ser um problema de {self.issue_type}.

Responda ao cliente com empatia e faça esta pergunta específica para iniciar a investigação:
- {investigation_questions[0]}

Explique brevemente que você precisa fazer algumas perguntas para entender melhor a situação e poder ajudar da melhor forma.
"""
            response = send_to_ai(initial_prompt)

        # Investigation questions (stages 1-4)
        elif 1 <= self.investigation_stage <= 4:
            # Store response to previous question
            question_keys = ["vizinhos", "disjuntores", "disjuntor_geral", "danos_visíveis"]
            self.investigation_responses[question_keys[self.investigation_stage-1]] = user_input

            # Get next question or conclude investigation
            investigation_questions = INVESTIGATION_QUESTIONS.get(
                self.issue_type, INVESTIGATION_QUESTIONS["DEFAULT"]
            )

            if self.investigation_stage < 4:
                next_question = investigation_questions[self.investigation_stage]

                next_prompt = f"""
O cliente respondeu: "{user_input}"

Continue a investigação de forma conversacional. Agradeça pela resposta e faça a próxima pergunta:
- {next_question}
"""
                response = send_to_ai(next_prompt)

In [5]:
import requests

# Configurações da API
API_KEY = "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"  # Sua chave do Groq
MODEL = "llama3-70b-8192"  # Modelo Llama 3 70B
URL = "https://api.groq.com/openai/v1/chat/completions"

def atendimento_elektro():
    print("\n🔵 Bem-vindo ao Atendimento Neoenergia Elektro! Como posso ajudar você hoje?")
    print("(Digite 'sair' a qualquer momento para encerrar o atendimento)\n")

    historico = [
        {
            "role": "system",
            "content": """Você é um atendente virtual da Neoenergia Elektro, especializado em resolver problemas de fornecimento de energia.
            Seu papel é identificar o problema do cliente com empatia e clareza, coletando informações precisas para resolver a situação.
            Sempre seja educado, paciente e mostre preocupação genuína com a situação do cliente.
            Pergunte de forma clara para identificar o tipo de problema (falta de energia, risco de morte, defeitos, etc.) e subtipo específico.
            Para problemas graves como 'risco de morte' ou 'emergência médica', priorize imediatamente e informe que acionará a equipe de emergência.
            Para outros problemas, explique o procedimento padrão e o prazo estimado para resolução."""
        }
    ]

    while True:
        # Pega a mensagem do cliente
        mensagem_cliente = input("Cliente: ")

        if mensagem_cliente.lower() in ["sair", "exit"]:
            print("\n🟡 Obrigado por entrar em contato com a Neoenergia Elektro. Tenha um bom dia!")
            break

        # Adiciona ao histórico
        historico.append({"role": "user", "content": mensagem_cliente})

        try:
            # Envia para a API
            resposta = requests.post(
                URL,
                headers={"Authorization": f"Bearer {API_KEY}"},
                json={
                    "model": MODEL,
                    "messages": historico,
                    "temperature": 0.5  # Menor variação para respostas mais consistentes
                }
            )

            # Processa a resposta
            if resposta.status_code == 200:
                resposta_ia = resposta.json()["choices"][0]["message"]["content"]
                print(f"\nAtendente Elektro: {resposta_ia}")
                historico.append({"role": "assistant", "content": resposta_ia})
            else:
                print(f"\n🔴 Desculpe, houve um erro em nosso sistema. Por favor, tente novamente mais tarde.")

        except Exception as e:
            print(f"\n🔴 Desculpe, estamos com problemas técnicos. Por favor, tente novamente ou ligue para nosso SAC: 0800 701 0102")

# Inicia o atendimento
if __name__ == "__main__":
    atendimento_elektro()


🔵 Bem-vindo ao Atendimento Neoenergia Elektro! Como posso ajudar você hoje?
(Digite 'sair' a qualquer momento para encerrar o atendimento)

Cliente: então meu amigo eu estou com um problema aqui que o meu chuveiro não está ligando, o restante da casa está tudo normal mas o chuveiro não esquenta

Atendente Elektro: Peço desculpas pelo incoveniente, senhor! Estou aqui para ajudá-lo a resolver o problema com o chuveiro. Posso imaginar como é frustrante não ter água quente para o seu banho.

Para melhor entender a situação, posso perguntar algumas coisas? Você notou se há alguma luz ou indicação de funcionamento no chuveiro? Ou se há algum som ou barulho quando você tenta ligá-lo?

Além disso, você já verificou se o problema é apenas com o chuveiro ou se há alguma outra parte da casa que também esteja afetada? Por exemplo, se as luzes ou outros aparelhos elétricos estão funcionando normalmente?

Com essas informações, posso começar a investigar o problema e encontrar a solução mais adequa

In [ ]:
import requests
from datetime import datetime

# Configurações da API Groq
API_KEY = "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"
MODEL = "llama3-70b-8192"
URL = "https://api.groq.com/openai/v1/chat/completions"

# Dicionário de problemas e classificações
PROBLEMAS = {
    "FALTA DE ENERGIA": [
        "FALTA DE ENERGIA GERAL", "FALTA DE ENERGIA", "FALTA DE FASE",
        "SIGFI FALTA DE ENERGIA FOTOVOLTAICO", "FALTA DE FASE GERAL",
        "Falta de Energia em Medidor Eletronico", "Falta de Fase em Medidor Eletrônico"
    ],
    "RISCO DE MORTE": [
        "OUTROS", "FIO PARTIDO NA REDE PRIMARIA", "FIO PARTIDO NO RAMAL DE SERVICO",
        "FIO PARTIDO NA REDE SECUNDARIA", "POSTE ABALROADO SEM FIO PARTIDO",
        "POSTE ABALROADO COM FIO PARTIDO"
    ],
    "PROVAVEL DEFEITO FALHA": [
        "FURTO FIACÃO/PADRÃO", "FALHA EM RAMAL DE SERVICO", "FIO DANDO FOGO NO RAMAL DE SERVICO",
        "FALHA EM MEDIDOR", "OBJETO ESTRANHO NA REDE SECUNDARIA", "OBJETO ESTRANHO NA REDE PRIMARIA",
        "POSTE PADRÃO QUEBRADO", "FIO DANDO FOGO NA REDE PRIMARIA", "FIO DANDO FOGO COM ARVORE NA REDE",
        "FURTO EQUIPAMENTO", "ELO FUSIVEL QUEIMADO/CAIDO", "FIO DANDO FOGO NA REDE SECUNDARIA",
        "CHOQUE EM INSTALAÇÃO", "FALHA EM QUICK-LAG", "FURTO REDE", "INVERSÃO DE FASE"
    ],
    "NIVEIS DE TENSÃO": [
        "TENSÃO BAIXA", "OSCILAÇÃO DE TENSÃO", "TENSÃO ALTA"
    ],
    "OUTROS": [
        "TRANSFORMADOR"
    ],
    "EMERGENCIA MEDICA": [
        "HOSPITAL COM PROBLEMA", "EQUIPAMENTO SUPORTE A VIDA"
    ]
}

def obter_saudacao():
    """Retorna a saudação conforme horário atual"""
    hora = datetime.now().hour
    if 5 <= hora < 12:
        return "Bom dia"
    elif 12 <= hora < 18:
        return "Boa tarde"
    return "Boa noite"

def obter_mensagem_fechamento():
    """Retorna mensagem padrão de encerramento"""
    return """
Agradecemos seu contato! Sempre que precisar estamos disponíveis nos nossos canais:

💻 Site: https://www.neoenergia.com/web/sp

📱 Aplicativo:
Android → https://bit.ly/NELK-Android
IOS → https://bit.ly/IOS-NELK

📞 Teleatendimento: 116
Deficiente auditivo ou de fala | 0800 701 01 55
Atendimento fora da área de concessão | 0800 276 0116

Conte sempre com a gente!
"""

def classificar_problema(descricao):
    """Classifica o problema em tipo e subtipo"""
    descricao = descricao.upper()
    for tipo, subtipos in PROBLEMAS.items():
        for subtipo in subtipos:
            if subtipo in descricao:
                return tipo, subtipo

    # Fallback para palavras-chave
    if "FALTA" in descricao and "ENERGIA" in descricao:
        return "FALTA DE ENERGIA", "FALTA DE ENERGIA"
    elif "FOGO" in descricao or "INCÊNDIO" in descricao:
        return "RISCO DE MORTE", "FIO DANDO FOGO NA REDE PRIMARIA"
    elif "CHOQUE" in descricao:
        return "PROVAVEL DEFEITO FALHA", "CHOQUE EM INSTALAÇÃO"
    elif "TENSÃO" in descricao or "VOLTAGEM" in descricao:
        return "NIVEIS DE TENSÃO", "OSCILAÇÃO DE TENSÃO"

    return "OUTROS", "OUTROS"

def verificar_problema_interno(descricao):
    """Verifica se problema é na instalação do cliente"""
    descricao = descricao.lower()
    indicadores = [
        "minha casa", "minha instalação", "meu disjuntor", "meu medidor",
        "apenas aqui", "só na minha casa", "circuito interno", "fiação interna",
        "disjuntor caiu", "queimou meu equipamento", "meu chuveiro", "minhas lâmpadas"
    ]
    return any(ind in descricao for ind in indicadores)

def gerar_resposta(tipo, subtipo, problema_interno):
    """Gera resposta adequada ao tipo de problema"""
    if problema_interno:
        return {
            "resposta": "Entendi que o problema parece ser na sua instalação interna. Recomendamos contatar um eletricista qualificado. Posso ajudar em algo mais?",
            "gerar_chamado": False
        }

    respostas = {
        "FALTA DE ENERGIA": {
            "mensagem": "Entendi que há falta de energia. Vou registrar um chamado para nossa equipe verificar.",
            "urgencia": "ALTA"
        },
        "RISCO DE MORTE": {
            "mensagem": "Situação de risco identificada! Equipe será acionada com urgência. Mantenha distância.",
            "urgencia": "URGENTE"
        },
        "PROVAVEL DEFEITO FALHA": {
            "mensagem": "Relato de defeito na rede entendido. Registrarei um chamado técnico.",
            "urgencia": "MEDIA"
        },
        "NIVEIS DE TENSÃO": {
            "mensagem": "Problema de tensão identificado. Registrarei um chamado para ajuste.",
            "urgencia": "MEDIA"
        },
        "EMERGENCIA MEDICA": {
            "mensagem": "Emergência médica! Acionaremos prioridade máxima.",
            "urgencia": "URGENTE"
        },
        "OUTROS": {
            "mensagem": "Entendi seu relato. Vou registrar um chamado para verificação.",
            "urgencia": "BAIXA"
        }
    }

    resposta = respostas.get(tipo, respostas["OUTROS"])
    return {
        "resposta": resposta["mensagem"],
        "gerar_chamado": True,
        "tipo": tipo,
        "subtipo": subtipo,
        "urgencia": resposta["urgencia"]
    }

def enviar_para_groq(mensagem):
    """Envia mensagem para API Groq e retorna resposta"""
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": MODEL,
        "messages": [
            {
                "role": "system",
                "content": "Você é um atendente virtual da Neoenergia Elektro. Seja empático e objetivo. Responda em português brasileiro de forma clara e curta."
            },
            {
                "role": "user",
                "content": mensagem
            }
        ],
        "temperature": 0.7,
        "max_tokens": 150
    }

    try:
        response = requests.post(URL, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return f"Desculpe, estou com dificuldades técnicas. Por favor, tente novamente mais tarde."

# Fluxo principal de atendimento
if __name__ == "__main__":
    print(f"{obter_saudacao()}! Sou o assistente virtual da Neoenergia Elektro. Como posso ajudar?")
    print("(Digite 'sair' para encerrar)")

    while True:
        mensagem = input("\nCliente: ").strip()

        if mensagem.lower() == 'sair':
            print(obter_mensagem_fechamento())
            break

        # Processa mensagem do cliente
        problema_interno = verificar_problema_interno(mensagem)
        tipo, subtipo = classificar_problema(mensagem)
        resposta = gerar_resposta(tipo, subtipo, problema_interno)

        print("\nAtendente:", resposta["resposta"])
        if resposta["gerar_chamado"]:
            print(f"(Chamado: {tipo} - {subtipo} | Urgência: {resposta['urgencia']})")

        # Resposta complementar da IA
        print("Atendente:", enviar_para_groq(mensagem))

Boa tarde! Sou o assistente virtual da Neoenergia Elektro. Como posso ajudar?
(Digite 'sair' para encerrar)

Cliente: Olá eu  estou com um problema, estou sem energia aqui em casa e pelo que eu vi tem uma chave lá no poste que caiu

Atendente: Entendi seu relato. Vou registrar um chamado para verificação.
(Chamado: OUTROS - OUTROS | Urgência: BAIXA)
Atendente: Desculpe, estou com dificuldades técnicas. Por favor, tente novamente mais tarde.

Cliente: então meu amigo eu estou com um problema aqui que o meu chuveiro não está ligando, o restante da casa está tudo normal mas o chuveiro não esquenta

Atendente: Entendi que o problema parece ser na sua instalação interna. Recomendamos contatar um eletricista qualificado. Posso ajudar em algo mais?
Atendente: Poxa, que chato! Não ter água quente no chuveiro pode ser muito inconveniente. 

Para ajudá-lo, gostaria de saber mais algumas coisas. Você verifica se o botão do chuveiro está ligado e se a chave de luz do banheiro está funcionando corr